In [6]:
import numpy as np
from kernel_functions import *
from string_embeddings import *

## Loading data
***

In [7]:
X = np.loadtxt('data/Xtr.csv', skiprows=1, usecols=(1,), dtype=str, delimiter=',')
y = np.loadtxt('data/Ytr.csv', skiprows=1, usecols=(1,), dtype=int, delimiter=',')
X_validation = np.loadtxt('data/Xte.csv', skiprows=1, usecols=(1,), dtype=str, delimiter=',')

X_full = np.hstack([X, X_validation])
y = 2*y - 1.

In [8]:
X_vectors = np.loadtxt('data/Xtr_mat100.csv')
X_validation_vectors = np.loadtxt('data/Xte_mat100.csv')

## Pre-computing Gram matrices
***

In [9]:
embeddings_full_data, kernels_full_data = {}, {}

In [10]:
# kmer_sizes = np.array([5, 6, 7, 8, 9, 10, 11])
kmer_sizes = [10, 11, 12]

In [11]:
%%time
print('Computing spectral embeddings')
for k in kmer_sizes:
    embedding_name = 'spectral k={}'.format(k)
    print(embedding_name)
    embeddings_full_data[embedding_name] = kmer_decomposition(X_full, k)
embeddings_full_data

Computing spectral embeddings
spectral k=10
spectral k=11
spectral k=12
CPU times: user 38.5 s, sys: 525 ms, total: 39 s
Wall time: 39.4 s


{'spectral k=10': <3000x1048576 sparse matrix of type '<class 'numpy.float64'>'
 	with 274552 stored elements in Compressed Sparse Row format>,
 'spectral k=11': <3000x4194304 sparse matrix of type '<class 'numpy.float64'>'
 	with 271822 stored elements in Compressed Sparse Row format>,
 'spectral k=12': <3000x16777216 sparse matrix of type '<class 'numpy.float64'>'
 	with 269014 stored elements in Compressed Sparse Row format>}

In [12]:
%%time
print('Computing spectral embeddings, reverse complement')
for k in kmer_sizes:
    embedding_name = 'spectral reverse k={}'.format(k)
    print(embedding_name)
    embeddings_full_data[embedding_name] = kmer_decomposition(X_full, k, reverse=True)
embeddings_full_data

Computing spectral embeddings, reverse complement
spectral reverse k=10
spectral reverse k=11
spectral reverse k=12


NameError: name 'embeddings_X' is not defined

In [13]:
%%time
print('Computing mismatch embeddings')
for k in kmer_sizes:
    embedding_name = 'mismatch k={}'.format(k)
    print(embedding_name)
    embeddings_full_data.setdefault(
        embedding_name, kmer_decomposition_mismatch(X_full, k)
    )

Computing mismatch embeddings
mismatch k=10
mismatch k=11
mismatch k=12
CPU times: user 11min 34s, sys: 8.33 s, total: 11min 42s
Wall time: 12min 41s


In [14]:
print('Computing spectral kernels')
for k in kmer_sizes:
    name = 'spectral k={}'.format(k)
    print(name)
    K = linear_kernel(embeddings_full_data[name], embeddings_full_data[name])
    kernels_full_data.setdefault(name, K)

Computing spectral kernels
spectral k=10
spectral k=11
spectral k=12


In [15]:
print('Computing spectral reverse kernels')
for k in kmer_sizes:
    name = 'spectral reverse k={}'.format(k)
    print(name)
    embed = embeddings_full_data[name]
    K = linear_kernel(embed, embed)
    kernels_full_data.setdefault(name, K)

Computing spectral reverse kernels
spectral reverse k=10
spectral reverse k=11
spectral reverse k=12


In [16]:
for key, x in embeddings_full_data.items():
    print(key, x.shape[0])

spectral k=10 3000
spectral k=11 3000
spectral k=12 3000
spectral reverse k=10 3000
spectral reverse k=11 3000
spectral reverse k=12 3000
mismatch k=10 3000
mismatch k=11 3000
mismatch k=12 3000


In [17]:
print('Computing mismatch kernels')
for k in kmer_sizes:
    name = 'mismatch k={}'.format(k)
    print(name)
    embed = embeddings_full_data[name], embeddings_full_data['spectral k={}'.format(k)]
    kernels_full_data[name] = mismatch_kernel(embed, embed)

Computing mismatch kernels
mismatch k=10
mismatch k=11
mismatch k=12


In [18]:
n_train = len(y)
embeddings_X = {
    k: v[:n_train] for k, v in embeddings_full_data.items()
}
kernels_X = {
    k: v[:n_train, :n_train] for k, v in kernels_full_data.items()
}

## Model Selection
***

In [19]:
from sklearn.model_selection import train_test_split

np.random.seed(1)
train_indices, test_indices = train_test_split(np.arange(len(y)))
# X_train_kmer, X_test_kmer, y_train, y_test = train_test_split(X_kmer, y)

In [20]:
def evaluate_kernel(clf, K):
    K_train = K[train_indices][:, train_indices]
    K_test = K[test_indices][:, train_indices]
    clf.fit_K(K_train, y[train_indices])
    y_fit = clf.predict_K(K_train)
    y_pred = clf.predict_K(K_test)
    print('Training accuracy: {:.1%}'.format((y_fit == y[train_indices]).mean()))
    print('Validation accuracy: {:.1%}'.format((y_pred == y[test_indices]).mean()))
    
def evaluate_X(clf, X):
    X_train, X_test = X[train_indices], X[test_indices]
    clf.fit(X_train, y[train_indices])
    y_fit = clf.predict(X_train)
    y_pred = clf.predict(X_test)
    print('Training accuracy: {:.1%}'.format((y_fit == y[train_indices]).mean()))
    print('Validation accuracy: {:.1%}'.format((y_pred == y[test_indices]).mean()))
    
def normalize(K):
    norms = np.sqrt(K[np.diag_indices_from(K)])
    return (K / norms[:, None]) / norms[None, :]

In [22]:
from kernel_ridge import KernelRidgeClassifier

lambd = 1.
KRC_linear = KernelRidgeClassifier(lambd=lambd)

evaluate_kernel(KRC_linear, kernels_X['spectral k=10'])

Training accuracy: 99.0%
Validation accuracy: 66.6%


In [23]:
lambd = .01
KRC_linear = KernelRidgeClassifier(lambd=lambd)
K = normalize(kernels_X['spectral k=10'])
evaluate_kernel(KRC_linear, K)

Training accuracy: 99.0%
Validation accuracy: 66.8%


In [24]:
lambd = 1.
KRC_linear = KernelRidgeClassifier(lambd=lambd)
evaluate_kernel(KRC_linear, normalize(kernels_X['mismatch k=12']))

Training accuracy: 98.7%
Validation accuracy: 71.6%


In [25]:
lambd = .001
KRC_linear = KernelRidgeClassifier(lambd=lambd)
K = normalize(kernels_X['mismatch k=11'])
evaluate_kernel(KRC_linear, K)

Training accuracy: 99.3%
Validation accuracy: 73.4%


In [27]:
lambd = .2
KRC_linear = KernelRidgeClassifier(lambd=lambd)

K = kernels_X['mismatch k=10'] + kernels_X['mismatch k=11'] + kernels_X['spectral k=10']
evaluate_kernel(KRC_linear, K)

Training accuracy: 99.5%
Validation accuracy: 73.2%


In [30]:
lambd = .001
KRC_linear = KernelRidgeClassifier(lambd=lambd)

Ks = [
    kernels_X['mismatch k=10'],
    kernels_X['mismatch k=11'],
    kernels_X['spectral k=10'],
   ]
K = np.mean([1+normalize(Ki) for Ki in Ks], axis=0)
evaluate_kernel(KRC_linear, K)

Training accuracy: 99.4%
Validation accuracy: 73.0%


In [32]:
from kernel_svm import KernelSVM
C = 1.
SVM_linear = KernelSVM(C=C)

K = normalize(kernels_X['spectral reverse k=10'])
%time evaluate_kernel(SVM_linear, K)

Training accuracy: 99.6%
Validation accuracy: 66.8%
CPU times: user 6.58 s, sys: 135 ms, total: 6.71 s
Wall time: 3.68 s


In [54]:
from kernel_ridge import KernelLogisticRegression

lambd = 1.
KLR = KernelLogisticRegression(lambd=lambd)

K = normalize(kernels_X['mismatch k=10'])
%time evaluate_kernel(KLR, K)

Training accuracy: 92.4%
Validation accuracy: 70.8%
CPU times: user 1.69 s, sys: 21.5 ms, total: 1.71 s
Wall time: 901 ms


## Ensemble of best models
***

In [112]:
def ensemble(model_specs, train_indices, test_indices):
    y_fit_mean = np.zeros_like(y[train_indices])
    y_pred_mean = np.zeros(len(test_indices))
    for model_spec in model_specs:
        clf = model_spec['classifier']
        K = model_spec['kernel matrix']
        K_train = K[train_indices, :n_train][:n_train, train_indices]
        K_test = K[test_indices, :n_train][:n_train, train_indices]
        
        clf.fit_K(K_train, y[train_indices])

        y_fit = clf.decision_function_K(K_train)
        y_fit_mean += y_fit #/ y_fit.std()
        y_pred = clf.decision_function_K(K_test)
        y_pred_mean += y_pred #/ float(y_pred.std())
    y_fit = np.where(y_fit_mean < 0, -1, 1)
    y_pred = np.where(y_pred_mean < 0, -1, 1)
    
    return y_fit, y_pred

In [123]:
models = []

models.append({
         'classifier': KernelRidgeClassifier(lambd=1.),
         'kernel matrix': normalize(kernels_X['mismatch k=11']),
     })
models.append({
         'classifier': KernelRidgeClassifier(lambd=1.),
         'kernel matrix': normalize(kernels_X['mismatch k=12']),
     })
models.append({
         'classifier': KernelLogisticRegression(lambd=.1),
         'kernel matrix': normalize(kernels_X['mismatch k=12']),
     })
models.append({
         'classifier': KernelLogisticRegression(lambd=1.),
         'kernel matrix': normalize(kernels_X['mismatch k=11']),
     })
#models.append({
#         'classifier': KernelSVM,
#         'params': {'C': 10.},
#         'kernel matrix': normalize(kernels_X['spectral k=8']),
#     })
# models.append({
#          'classifier': KernelSVM(C=10.),
#          'kernel matrix': normalize(kernels_X['spectral k=8']),
#      })

In [124]:
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=5, shuffle=True, random_state=2)
#kf.get_n_splits(np.arange(len(y)))
train_accuracy = []
test_accuracy = []
for train_indices, test_indices in kf.split(np.arange(len(y))):
    y_fit, y_pred = ensemble(models, train_indices, test_indices)
    train_accuracy.append((y_fit == y[train_indices]).mean())
    test_accuracy.append((y_pred == y[test_indices]).mean())
    
print('Mean Training accuracy: {:.1%}'.format(np.mean(train_accuracy)))
print('Mean Validation accuracy: {:.1%}'.format(np.mean(test_accuracy)))

Mean Training accuracy: 98.7%
Mean Validation accuracy: 69.0%


In [48]:
models[0]['classifier'].__dict__

{'lambd': 0.001,
 'kernel_name': 'linear',
 'kernel_function_': <function kernel_functions.linear_kernel(X1, X2)>,
 'kernel_parameters': {},
 'y_train': array([ 1., -1.,  1., ...,  1.,  1.,  1.]),
 'n_iter': 2,
 'alpha': array([ 0.13834226, -0.14330538,  0.14623698, ...,  0.14563596,
         0.14821527,  0.13490521])}

In [50]:
models[0]['classifier'].alpha.shape

(1600,)

In [125]:
final_models =[
    {
         'classifier': kernel_ridge.KernelRidgeClassifier(lambd=1.),
         'kernel matrix': normalize(kernels_full_data['mismatch k=11']),
    },
    {
        'classifier': KernelRidgeClassifier(lambd=1.),
        'kernel matrix': normalize(kernels_full_data['mismatch k=12']),
    },
    {
         'classifier': KernelLogisticRegression(lambd=.1),
         'kernel matrix': normalize(kernels_full_data['mismatch k=12']),
    },
    {
         'classifier': KernelLogisticRegression(lambd=1.),
         'kernel matrix': normalize(kernels_full_data['mismatch k=11']),
    },
]

In [126]:
y_fit, y_validation = ensemble(final_models, np.arange(n_train), np.arange(n_train, len(X_full)))
(y_fit == y).mean()

0.9805

In [127]:
y_validation[y_validation == -1] = 0
#y_validation *= 0
y_save = np.vstack([np.arange(len(y_validation)), y_validation]).T
y_save[:10]

array([[0, 1],
       [1, 1],
       [2, 0],
       [3, 0],
       [4, 0],
       [5, 1],
       [6, 1],
       [7, 0],
       [8, 0],
       [9, 1]])

In [128]:
np.savetxt('mismatch_11_12.csv', y_save,
           delimiter=',', header='Id,Bound', fmt='%i', comments='')